<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [67]:
# Импорт основных библиотек для работы с БД и данными
import pandas as pd
import psycopg2

# Для общей эстетики подавляем предупреждения об ошибках
import warnings
warnings.filterwarnings('ignore')

In [69]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [70]:
# Обобщеный комментарий для запросов SQL 
# SELECT Count(*) - запрос количества записей
# FROM  - выборка из требуемой таблицы
vacancies_count = f'''select 
           count(id) AS vacancies_count
           from public.vacancies
        '''

In [71]:
df = pd.read_sql_query(vacancies_count, connection)
df
# результат запроса - в нашей базе 49197 вакансий

,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [72]:
employers_count = f'''select 
           count(id) AS employers_count
           from public.employers
        '''

In [73]:
df = pd.read_sql_query(employers_count, connection)
df
# результат запроса - в нашей базе 23501 работодатель

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [74]:
areas_count = f'''select 
           count(id) AS areas_count
           from public.areas
        '''

In [75]:
df = pd.read_sql_query(areas_count, connection)
df
# результат запроса - в нашей базе 1362 региона

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [76]:
industries_count = f'''select 
           count(id) AS industries_count
           from public.industries
        '''

In [77]:
df = pd.read_sql_query(industries_count, connection)
df
# результат запроса - в нашей базе 294 сферы деятельности

,industries_count
0,294


***

 Выводы по предварительному анализу данных:
  
 Получена размерность таблиц, с помощью которых будет сделано дальнейшее исследование и анализ данных. В базе данных представлены 49197 вакансий от 23501 работодателя, что говорит о том, что некоторые работодатели открыли несколько вакансий . География вакансий довольна обширна и представлена 1362 регионом. Диапазон сфер деятельности также довольно широк и представлен 294 направлениями.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [78]:
# выведем для анализа 10 лидеров вместо 5
cnt_vac_areas = f'''select 
           count(v.id) AS vacancies_count,
           a.name AS area_name
           from public.vacancies v
           JOIN public.areas a ON v.area_id=a.id
           GROUP BY a.name
           ORDER BY 1 DESC
           LIMIT 10
        '''

In [79]:
df = pd.read_sql_query(cnt_vac_areas, connection)
df
# Пятёрка лидеров по количеству вакансий: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы 

,vacancies_count,area_name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
5,1698,Екатеринбург
6,1670,Нижний Новгород
7,1415,Казань
8,1301,Краснодар
9,1144,Самара


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [80]:
salary_not_null = f'''select 
           count(id)
           from public.vacancies 
           WHERE (salary_from IS NOT NULL) OR (salary_to) IS NOT NULL
        '''

In [81]:
df = pd.read_sql_query(salary_not_null, connection)
df
# результат запроса-24073 вакансии, у которых заполнено хотя бы одно поле с зарплатой

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [82]:
# Применяется преобразование к целочисленному типу
salary_average = f'''select 
           AVG(salary_from)::integer AS avg_min,
           AVG(salary_to)::integer AS avg_max
           from public.vacancies 
        '''

In [83]:
df = pd.read_sql_query(salary_average, connection)
df
# результат запроса - нижняя граница зарплатной вилки - 71065, верхняя - 110537

,avg_min,avg_max
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [84]:
vac_schedule = f'''select 
           COUNT(id),
           schedule,
           employment
           from public.vacancies 
           GROUP BY schedule, employment
           ORDER BY 1 DESC
        '''

In [85]:
df = pd.read_sql_query(vac_schedule, connection)
df

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [86]:
required_exp = f'''select 
           experience AS required_experience,
           count(id) AS vacancies_count
           from public.vacancies
           WHERE experience IS NOT NULL 
           GROUP BY experience
           ORDER BY 2 ASC
        '''

In [87]:
df = pd.read_sql_query(required_exp, connection)
df

,required_experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий:
1. Представлены не только вакансии в России, но и в других странах. Самое большое количество вакансий расположено в столицах и крупных городах. Лидируют Москва и Санкт-Петербург. На долю первых 6-ти крупных городов (Москва, Санкт-Петербург, Минск, Новосибирск, Алматы, Екатеринбург) приходится треть всех вакансий;
2. У половины вакансий не заполнена заработная плата. Вероятно, заработная плата работодателем назначается либо по результатам собеседования, либо озвучивается лично соискателю (коммерческая тайна). В некоторых случаях не указана одна из границ заработной платы, что скорее всего указывает на фиксированную ставку.
3. В среднем работодатели указывают зарплатную вилку от 71 до 110 тысяч. Веротяно, в регионах суммы гораздо меньше, на средний показатель в большей степени влияет зарплатная вилка вакансий в крупных городах, а также вакансии в других странах;
4. По распорядку и типу занятости лидирует "Полный день/Полная занятость", на втором месте - "Удаленная работа" (пока в ~4.5 раза меньше). Это закономерно, работодатели считают работу по полному графику более эффективной, нежели удаленную, а также некоторые сферы деятельности вовсе не подразумевают удалённый график. К сожалению, в базе данных не фигурируют даты, чтобы отследить рост/спад и тенденции.
5. Больше половины вакансий рассчитаны на кандидатов с небольшим опытом (1-3 года). В 29% хотят видеть более опытных кандидатов (3-6 лет). Кандидатов без опыта готовы рассмотреть только в 15% вакансий, так как на подготовку таких работников требуется слишком много ресурсов. Столь невысокие требования могут быть связаны с тем, что soft skills становятся для работодателя важнее, чем профессиональный опыт, либо нежеланием/невозможностью платить соискателю слишком высокий уровень зарплаты. 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [88]:
prevalent_emp = f'''(select 
           e.name AS company_name,
           count(v.id) AS vacancies_count
           FROM public.vacancies AS v
           JOIN public.employers AS e ON v.employer_id = e.id
           GROUP BY e.name
           ORDER BY 2 DESC
           LIMIT 1)
           UNION ALL
           (select
           ee.name,
           count(vv.id)
           FROM public.vacancies AS vv
           JOIN public.employers AS ee ON vv.employer_id = ee.id
           GROUP BY ee.name
           ORDER BY 2 DESC
           OFFSET 4
           LIMIT 1)
        '''
        

In [89]:
df = pd.read_sql_query(prevalent_emp, connection)
df
# На первом месте - Яндекс, на пятом месте - Газпром нефть.

,company_name,vacancies_count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [90]:
area_distribution = f'''select 
           a.name,
           COUNT(e.id) AS emp,
           COUNT(v.id) AS vac
           FROM public.areas a
           LEFT JOIN public.employers e ON e.area=a.id
           LEFT JOIN public.vacancies v ON v.area_id=a.id
           WHERE v.id IS NULL
           GROUP BY a.name
           ORDER BY 2 DESC
           LIMIT 10
        '''

In [91]:
df = pd.read_sql_query(area_distribution, connection)
df

,name,emp,vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [92]:
employers_area = f'''select
           e.name AS employer_name,
           COUNT(DISTINCT v.area_id) AS areas_count
           from public.employers e
           LEFT JOIN public.vacancies v ON v.employer_id=e.id
           GROUP BY e.name
           ORDER BY 2 DESC
           LIMIT 10
        '''

In [93]:
df = pd.read_sql_query(employers_area, connection)
df

,employer_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [94]:
employers_industries = f'''select 
           COUNT(e.id) AS employers_count
           from public.employers e
           LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id
           WHERE industry_id IS NULL
        '''

In [95]:
df = pd.read_sql_query(employers_industries, connection)
df
# результат запроса - 8419

,employers_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [96]:
four_industries_employers = f'''select 
           e.name AS emp_name,
           COUNT(ei.industry_id) AS ind_id
           from public.employers e
           LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id
           GROUP BY e.name
           HAVING COUNT(ei.industry_id)=4
           ORDER BY 1
           OFFSET 2 LIMIT 1 
        '''

In [97]:
df = pd.read_sql_query(four_industries_employers, connection)
df
# результат запроса - 2ГИС

,emp_name,ind_id
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [98]:
software_employers = f'''select 
           COUNT(ei.employer_id) AS employers_count
           from public.employers_industries ei
           LEFT JOIN public.industries i ON ei.industry_id=i.id
           WHERE i.name LIKE 'Разработка программного обеспечения'
        '''

In [99]:
df = pd.read_sql_query(software_employers, connection)
df
# результат запроса - 3553

,employers_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [100]:
Yandex = f'''select 
           a.name AS a_name,
           COUNT(*) cnt
           from public.employers e
           JOIN public.vacancies v ON v.employer_id=e.id
           JOIN public.areas a ON a.id=v.area_id
           WHERE e.name='Яндекс' AND a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
           'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 
           'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
           GROUP BY a.id
           UNION ALL
           select
           'Total',
           COUNT(*) 
           from public.vacancies v
           JOIN public.employers e ON v.employer_id=e.id
           JOIN public.areas a ON a.id=v.area_id
           WHERE e.name='Яндекс' AND a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 
           'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 
           'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
           ORDER BY cnt
        '''

In [101]:
df = pd.read_sql_query(Yandex, connection)
df

,a_name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


In [102]:
# результат запроса - 17 строк, тотал - 485

***

Выводы по анализу работодателей:
1. Среди работодателей безоговорочным лидером по найму персонала является компания Яндекс (1933 вакансии, 8% всех вакансий). Количество размещённых вакансий этой компании в 4 раза превышаает количество вакансий следующего работодателя в рейтинге  - Ростелеком. Это говорит о том, что Яндекс динамично развивается и расширяет свой бизнес, в том числе и по географии (об этом свидетельствует тот факт, что Яндекс также является лидером по количеству регионов, в которых работодатели размещают свои вакансии).
2. В базе данных есть регионы в которых не размещены вакансии. На основе анализа можно выявить следующую закономерность: чем крупнее регион (страна, республика, город), тем больше в ней работодателей без размещённых вакансий. Это может быть связано с небольшими компаниями, которые обобщенно указывали наиболее крупный регион, либо с работодателями, которые предлагали удаленную работу. 
3. Вакансии в наибольшем количестве регионов представлены у таких крупных компаний, как Яндекс и Ростелеком.Это можно объяснить широкой сетью офисов (характерно для Ростелекома), а в случае Яндекса с предложением типа рабочего графика "Удалённая работа".
4. У 8419 работодателей (36%) не указана сфера деятельности. Это может негативно сказаться на результатах анализа сфер деятельности работодателей; А у некоторых компаний указано несколько сфер деятельности.
5. Разработкой программного обеспечения занимаются 3553 работодателя (15% среди всех работодателей и 24% из указавших сферу деятельности). 
6. Компания "Яндекс" предлагает вакансии в 181 регионе и в каждом из 16 городов-миллионников. Стоит отметить, что мы учли только российские города-миллионники, а в вакансиях имются и крупные города других стран. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [103]:
data_vacancies= f'''select 
           COUNT(id) AS cnt_vac
           from public.vacancies
           WHERE (lower(name)) LIKE '%data%' OR 
           (lower(name)) LIKE '%данн%'
        '''

In [104]:
df = pd.read_sql_query(data_vacancies, connection)
df
# результат запроса - 1771

,cnt_vac
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [105]:
data_science_vacancies= f'''select 
           COUNT(*) AS cnt_vac
           from public.vacancies v
           where (name ilike '%data scientist%'
   	       or name ilike '%data science%'
   	       or name ilike '%исследователь данных%'
   	       or (name like '%ML%' and name not ilike '%HTML%')
           or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
 	       and
 	       (name ilike '%junior%'
  	       or v.experience = 'Нет опыта'
  	       or v.employment='Стажировка')
        '''

In [106]:
df = pd.read_sql_query(data_science_vacancies, connection)
df
# результат запроса - 51

,cnt_vac
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [107]:
DS_SQL_vacancies= f'''select 
           COUNT(*) AS cnt_vac
           from public.vacancies v
           where (name ilike '%data scientist%'
   	       or name ilike '%data science%'
   	       or name ilike '%исследователь данных%'
   	       or (name like '%ML%' and name not ilike '%HTML%')
           or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
   	       )
 	       and
  	       (v.key_skills ILIKE '%postgres%'
  	       or key_skills ILIKE '%SQL%')
        '''

In [108]:
df = pd.read_sql_query(DS_SQL_vacancies, connection)
df
# результат запроса - 201

,cnt_vac
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [109]:
DS_Python_vacancies= f'''select 
           COUNT(*) AS cnt_vac
           from public.vacancies v
           where (name ilike '%data scientist%'
   	       or name ilike '%data science%'
   	       or name ilike '%исследователь данных%'
   	       or (name like '%ML%' and name not ilike '%HTML%')
           or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
   	       )
 	       and
  	       (v.key_skills ILIKE '%python%')
        '''

In [110]:
df = pd.read_sql_query(DS_Python_vacancies, connection)
df
# результат запроса - 351

,cnt_vac
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [111]:
DS_key_skills_vacancies= f'''select 
           round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2) as avg_ks
           from public.vacancies v
           where (name ilike '%data scientist%'
   	       or name ilike '%data science%'
   	       or name ilike '%исследователь данных%'
   	       or (name like '%ML%' and name not ilike '%HTML%')
           or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
   	       )
           and (key_skills IS NOT NULL)
        '''

In [112]:
df = pd.read_sql_query(DS_key_skills_vacancies, connection)
df
# результат запроса - 6.41

,avg_ks
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [113]:
DS_salary_range = f'''select 
           v.experience,
           round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) AS average_salary
           from public.vacancies v
           WHERE (v.name ILIKE '%data scientist%'
   	     OR v.name ILIKE '%data science%'
   	     OR v.name ILIKE '%исследователь данных%'
   	     OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
           OR v.name ILIKE '%machine learning%' OR v.name ILIKE '%машинн%обучен%'
           )
           AND (v.salary_from IS NOT NULL or v.salary_to IS NOT NULL)
           GROUP BY v.experience
        '''

In [114]:
df = pd.read_sql_query(DS_salary_range, connection)
df
# результат запроса - 243115

,experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

Выводы по предметному анализу:

1. Найденно 1771 вакансий связанных с данными. Вакансии для специалистов по Data Science  составляют менее 4% от общего числа вакансий. Что довольно мало, учитывая бурное развитие технологий обработки больших данных и искусственного интеллекта.

2. Для начинающих дата-сайентистов уровня "Junior" найдено всего 51 вакансий из 480 вакансий по DS (11%). Видимо, не все работодатели хотят или имеют ресурсы на обучение начинающих специалистов.

3. SQL указан в качестве ключевого навыка в 201 вакансиях DS из 480 (42%). Знания Python требуются в 351 вакансиях DS из 480 (73%). Таким образом, SQL и Python являются основополагающими и наиболее востребованными ключевыми навыками. В среднем работодатель хочет, чтобы соискатель обладал 6-7 компетенциями, то есть был высококвалифицированным специалистом. Данное обстоятельство может быть одной из причин небольшого числа вакансий для начинающего специалиста.

4. Средняя заработная плата для DS коррелирует с опытом кандидатов. Самая высокая средняя зарплата в DS у кандидатов с опытом более 3-х лет. Заработная плата кандидатов даже с опытом от 1 года (140 тыс) выше, чем верхняя вилка средней заработной платы по всем вакансиям, которую мы рассчитывали ранее (110 537 руб.). Вероятно, это связано с тем, что эта сфера требует много профессиональных компетенций


# Общий вывод по проекту

1. Проанализируем каких вакансий больше всего в нашей базе данных.

In [115]:
prevalent_vacancies = f'''
    SELECT
    name AS vacancy_name,
    COUNT(*) AS vacancy_count
    FROM public.vacancies
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
'''

In [116]:
df = pd.read_sql_query(prevalent_vacancies, connection)
df

,vacancy_name,vacancy_count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


Данный анализ демонстрирует, что наибольшее количество вакансий открыто в сфере информационных технологий. Это говорит о том, что сфера IT в данный момент наиболее динамично развивающаяся и крайне востребована и перспективна для потенциальных соискателей. 

2. Проанализируем средние заработные платы по регионам

In [117]:
Average_salary = (f'''
    SELECT
    a.name AS area_name,
    Round(Avg((Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2)) AS avg_salary, 
    Count(v.id) AS vacancies_count
    FROM public.areas a
    JOIN public.vacancies v ON v.area_id = a.id
    WHERE  (v.salary_to IS NOT NULL 
    OR v.salary_from IS NOT NULL)
    GROUP BY area_name
    HAVING COUNT(v.id) > 15
    ORDER BY 2 DESC
    LIMIT 20
''')

In [118]:
df = pd.read_sql_query(Average_salary, connection)
df

,area_name,avg_salary,vacancies_count
0,Турция,235435.0,19
1,Кипр,223736.0,39
2,Армения,219570.0,106
3,Тбилиси,207156.0,80
4,Батуми,199834.0,18
5,Иннополис,175882.0,17
6,Баку,155747.0,51
7,Москва,134140.0,1592
8,Магадан,117033.0,19
9,Зеленоград,116946.0,76


Как мы видим, самые высокие зарплаты предлагают в других странах, но количество таких вакансий невелико. Для более корректного анализа было бы лучше разделять регионы по странам.

Общие итоги:

1. Наибольшее число вакансий представлено в крупных городах. Только в Москве представлено 11% всех вакансий, а в Санкт-Петербурге еще 6%. Имеются вакансии в разных странах.

2. Заработные платы за рубежом в среднем выше, чем в России. А в крупных городах России выше, чем в других регионах.

3. Есть крупные работодатели, которые ищут специалистов во многих городах. Либо такие компании имеют филилалы по всей стране, либо готовы на удаленную работу. Например, Яндекс (152 регион), Ростелеком (181 регион). Эти компании также являются лидерами по числу вакансий 1933 и 491 соответственно (5% от всех вакансий).

4. Идеальный соискатель для работодателя - это работник, готовый к полной занятости и имеющий опыт от 1 до 6 лет.

5. IT - это быстро развивающаяся и наиболее перспективная сфера деятельности. Подавляющее большинство вакансий связаны с IT. 

6. Среди IT специалистов начинают требоваться специалисты по Data Science. Заработные платы таких специалистов одни из самых высоких. Однако, есть сложности со стартом в этом направлении, вакансий для начинающих специалистов уровня "Junior" не так много. Начинающему специалисту по DS необходимо знание SQL и Python, а также обладать дополнительными компетенциями (в среднем 6-7 ключевых навыков).

7. Затруднён анализ по сферам деятельности компаний. В представленной базе данных они часто уникальны, у работодателя может быть несколько сфер деятельности, а также у большого количества компаний вид деятельности вообще не указан.